In [1]:
import pandas as pd
import numpy as np
import re
import string
import csv
from collections import defaultdict

# For fuzzy matching
from fuzzywuzzy import fuzz
import Levenshtein
from jellyfish import jaro_winkler_similarity, soundex
import difflib

# For embedding-based matching
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# For NLP operations
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)

# For evaluation
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

#------------------------------------------------------------------------------
# DATA LOADING
#------------------------------------------------------------------------------

def load_data_from_excel(file_path="Acronym.xlsx"):
    """
    Load acronym data from the Excel file.
    
    Args:
        file_path (str): Path to the Excel file containing acronym data.
        
    Returns:
        tuple: (full_forms, test_data) where full_forms is a list of all full forms
               and test_data is a list of tuples (acronym, correct_full_form, context).
    """
    try:
        print(f"Loading data from {file_path}...")
        df = pd.read_excel(file_path)
        
        # Check if the required columns exist
        required_columns = ["Acronym", "Full Name"]
        if not all(col in df.columns for col in required_columns):
            print(f"Error: Excel file must contain columns {required_columns}")
            print(f"Found columns: {df.columns.tolist()}")
            return [], []
        
        # Extract unique full forms to use as our reference list
        full_forms = df["Full Name"].unique().tolist()
        
        # Create test data in the format (acronym, correct_full_form, context)
        test_data = []
        for _, row in df.iterrows():
            acronym = row["Acronym"]
            full_form = row["Full Name"]
            
            # Determine context based on the full form
            context = determine_context(full_form)
            
            test_data.append((acronym, full_form, context))
        
        print(f"Loaded {len(full_forms)} unique full forms and {len(test_data)} test cases.")
        return full_forms, test_data
        
    except Exception as e:
        print(f"Error loading data from Excel file: {str(e)}")
        return [], []

def determine_context(full_form):
    """
    Determine the context of a full form based on keywords.
    
    Args:
        full_form (str): The full form to analyze.
        
    Returns:
        str: The determined context or None if no context can be determined.
    """
    full_form_lower = full_form.lower()
    
    # Define context categories and their keywords
    context_keywords = {
        "finance": ["bank", "financial", "insurance", "investment", "provident", "securities", "credit"],
        "food": ["food", "restaurant", "coffee", "mcdonalds", "starbucks", "cafe"],
        "retail": ["retail", "store", "market", "walmart", "depot", "shopping", "supermarket", "pharmacy"],
        "technology": ["technology", "computer", "electronic", "digital", "software", "systems", "aws", "ibm", "hp"],
        "government": ["government", "municipal", "authority", "commission", "public", "national", "federal", "agency"],
        "transportation": ["automobile", "transport", "aviation", "airlines", "aerial", "travel", "road"],
        "healthcare": ["health", "medical", "hospital", "pharmaceutical", "laboratories", "medicine"],
        "education": ["education", "university", "school", "college", "institute", "research", "academic"],
        "entertainment": ["broadcasting", "entertainment", "media", "television", "radio", "sport"]
    }
    
    # Check each context category
    for context, keywords in context_keywords.items():
        if any(keyword in full_form_lower for keyword in keywords):
            return context
    
    return None

def create_example_data():
    """Create example data for demonstration and testing when no Excel file is available."""
    # Example full forms
    full_forms = [
        "McDonald's",
        "Bank of America",
        "Walmart",
        "Starbucks Coffee",
        "Microsoft",
        "Amazon",
        "Google",
        "Facebook",
        "Apple",
        "International Business Machines",
        "Intel Corporation",
        "CVS Pharmacy",
        "Target",
        "Coca-Cola",
        "PepsiCo",
        "Johnson & Johnson",
        "Procter & Gamble",
        "Municipal Corporation Delhi",
        "Artificial Intelligence",
        "Machine Learning",
        "Natural Language Processing",
        "Deep Learning",
        "Central Processing Unit",
        "Graphics Processing Unit",
        "Random Access Memory",
        "Hard Drive",
        "Solid State Drive",
        "7-Eleven",
        "Home Depot",
        "Walmart Supercenter"
    ]
    
    # Example test data
    test_data = [
        ("MCD", "McDonald's", "food"),
        ("BOFA", "Bank of America", "finance"),
        ("WM", "Walmart", "retail"),
        ("SBUX", "Starbucks Coffee", "food"),
        ("MSFT", "Microsoft", "technology"),
        ("AMZN", "Amazon", "retail"),
        ("GOOG", "Google", "technology"),
        ("FB", "Facebook", "technology"),
        ("AAPL", "Apple", "technology"),
        ("IBM", "International Business Machines", "technology"),
        ("INTC", "Intel Corporation", "technology"),
        ("CVS", "CVS Pharmacy", "retail"),
        ("TGT", "Target", "retail"),
        ("KO", "Coca-Cola", "food"),
        ("PEP", "PepsiCo", "food"),
        ("JNJ", "Johnson & Johnson", "health"),
        ("PG", "Procter & Gamble", "retail"),
        ("MCD GOV", "Municipal Corporation Delhi", "government"),
        ("AI", "Artificial Intelligence", "technology"),
        ("ML", "Machine Learning", "technology"),
        ("NLP", "Natural Language Processing", "technology"),
        ("DL", "Deep Learning", "technology"),
        ("CPU", "Central Processing Unit", "technology"),
        ("GPU", "Graphics Processing Unit", "technology"),
        ("RAM", "Random Access Memory", "technology"),
        ("HD", "Hard Drive", "technology"),
        ("SSD", "Solid State Drive", "technology"),
        ("McDnlds", "McDonald's", "food"),
        ("B of A", "Bank of America", "finance"),
        ("Wlmrt", "Walmart", "retail"),
        ("Strbcks", "Starbucks Coffee", "food"),
        ("Mcrsft", "Microsoft", "technology"),
        ("7-11", "7-Eleven", "retail"),
        ("7-Thirteen", "7-Eleven", "retail"),
        ("Seven Eleven", "7-Eleven", "retail"),
        ("Home Depot Inc", "Home Depot", "retail"),
        ("WLMRT", "Walmart Supercenter", "retail"),
        ("Wal-Mart", "Walmart Supercenter", "retail"),
        ("MD", "McDonald's", "food"),
        ("MLD", "McDonald's", "food")
    ]
    
    return full_forms, test_data

#------------------------------------------------------------------------------
# RULE-BASED MATCHING
#------------------------------------------------------------------------------

class RuleBasedMatcher:
    def __init__(self):
        """
        Initialize the rule-based matcher with a dictionary of known acronyms.
        This is the fastest matching component and handles exact matches.
        """
        # Initialize with known acronym mappings from the dataset
        self.acronym_dict = {
            # Food & Restaurants
            "MCD": "McDonald's",
            "MD": "McDonald's",
            "MLD": "McDonald's",
            "MCDNLDS": "McDonald's",
            "SBUX": "Starbucks Coffee",
            "STARBUCKS": "Starbucks Coffee",
            "KO": "Coca-Cola",
            "PEP": "PepsiCo",
            
            # Retail
            "WLMRT": "Walmart Supercenter",
            "WAL-MART": "Walmart Supercenter",
            "WM": "Walmart",
            "WMT": "Walmart",
            "TGT": "Target",
            "CVS": "CVS Pharmacy",
            "CONSUMER VALUE STORE PHARMACY": "CVS Pharmacy",
            "THE HOME DEPOT": "Home Depot",
            "HOME DEPOT INC": "Home Depot",
            "HD": "Home Depot",
            "7-ONE ONE": "7-Eleven",
            "7-THIRTEEN": "7-Eleven",
            "9-18": "7-Eleven",
            "7-12": "7-Eleven",
            "7-11": "7-Eleven",
            "SEVEN ELEVEN": "7-Eleven",
            
            # Finance
            "BOFA": "Bank of America",
            "BANK OF AMERICA CORP": "Bank of America",
            "BOA BANK": "Bank of America",
            "B OF A": "Bank of America",
            "NAB": "National Australia Bank",
            "CBA": "Commonwealth Bank of Australia",
            "ANZ": "Australia and New Zealand Banking Group",
            "WESTPAC": "Western Pacific Banking Corporation",
            "RBA": "Reserve Bank of Australia",
            "AMP": "Australian Mutual Provident Society",
            "HDFC": "Housing Development Finance Corporation",
            "ICICI": "Industrial Credit and Investment Corporation of India",
            
            # Technology
            "IBM": "International Business Machines",
            "MSFT": "Microsoft",
            "AAPL": "Apple",
            "GOOG": "Google",
            "AMZN": "Amazon",
            "FB": "Facebook",
            "INTC": "Intel Corporation",
            "HP": "Hewlett-Packard",
            "TCS": "Tata Consultancy Services",
            "AWS": "Amazon Web Services",
            "MYOB": "Mind Your Own Business",
            "CPU": "Central Processing Unit",
            "GPU": "Graphics Processing Unit",
            "RAM": "Random Access Memory",
            "HD": "Hard Drive",
            "SSD": "Solid State Drive",
            
            # Government & Organizations
            "MCD GOV": "Municipal Corporation Delhi",
            "ACCC": "Australian Competition and Consumer Commission",
            "ASIC": "Australian Securities and Investments Commission",
            "ATO": "Australian Taxation Office",
            "DFAT": "Department of Foreign Affairs and Trade",
            "EU": "European Union",
            "NASA": "National Aeronautics and Space Administration",
            
            # Other
            "AI": "Artificial Intelligence",
            "ML": "Machine Learning",
            "NLP": "Natural Language Processing",
            "DL": "Deep Learning",
            "JNJ": "Johnson & Johnson",
            "PG": "Procter & Gamble",
            "GE": "General Electric",
            "3M": "Minnesota Mining and Manufacturing",
            "BYD": "Build Your Dreams",
            "LG": "Life is Good",
            "BHP": "Broken Hill Proprietary Company",
            "CSL": "Commonwealth Serum Laboratories",
            "KPMG": "Klynveld Peat Marwick Goerdeler",
            "ABC": "Australian Broadcasting Corporation",
            "SBS": "Special Broadcasting Service",
            "AFL": "Australian Football League",
            "ASX": "Australian Securities Exchange",
            "FOXTEL": "Fox News Corporation & Telstra"
        }
        
    def match(self, acronym):
        """
        Match an acronym using the rule-based approach.
        
        Args:
            acronym (str): The acronym to match.
            
        Returns:
            tuple: (matched_full_form, confidence_score)
        """
        # Convert to uppercase to ensure case-insensitive matching
        acronym_upper = acronym.upper()
        
        # Check if the acronym exists in our dictionary
        if acronym_upper in self.acronym_dict:
            return self.acronym_dict[acronym_upper], 1.0  # Return match with 100% confidence
        
        return None, 0.0  # No match found, 0% confidence
    
    def add_acronym(self, acronym, full_form):
        """Add a new acronym to the dictionary."""
        self.acronym_dict[acronym.upper()] = full_form

#------------------------------------------------------------------------------
# FUZZY MATCHING
#------------------------------------------------------------------------------

class FuzzyMatcher:
    def __init__(self, known_full_forms):
        """
        Initialize the fuzzy matcher with a list of known full forms.
        This handles spelling variations and misspellings.
        
        Args:
            known_full_forms (list): List of known full forms to match against.
        """
        self.known_full_forms = known_full_forms
        self.threshold_high = 0.85  # High confidence threshold
        self.threshold_low = 0.60   # Low confidence threshold
        
    def match(self, acronym):
        """
        Match an acronym using fuzzy matching techniques.
        
        Args:
            acronym (str): The acronym to match.
            
        Returns:
            tuple: (matched_full_form, confidence_score, confidence_level)
        """
        best_match = None
        best_score = 0
        
        # Check similarity against each known full form
        for full_form in self.known_full_forms:
            # Try different fuzzy matching methods and take the highest score
            jw_score = jaro_winkler_similarity(acronym.lower(), full_form.lower())
            lev_ratio = Levenshtein.ratio(acronym.lower(), full_form.lower())
            token_sort_ratio = fuzz.token_sort_ratio(acronym, full_form) / 100.0
            
            # Take the highest score from the different methods
            score = max(jw_score, lev_ratio, token_sort_ratio)
            
            if score > best_score:
                best_match = full_form
                best_score = score
        
        # Decision based on confidence thresholds
        if best_score >= self.threshold_high:
            confidence = "high"
        elif best_score >= self.threshold_low:
            confidence = "medium"
        else:
            confidence = "low"
            
        return best_match, best_score, confidence

#------------------------------------------------------------------------------
# EMBEDDING-BASED MATCHING
#------------------------------------------------------------------------------

class EmbeddingMatcher:
    def __init__(self, known_full_forms, context_examples=None):
        """
        Initialize the embedding matcher with a list of known full forms and context examples.
        This handles context-aware matching and semantic understanding.
        
        Args:
            known_full_forms (list): List of known full forms to match against.
            context_examples (dict, optional): Dictionary mapping contexts to lists of examples.
        """
        # Load a pre-trained sentence transformer model
        self.model = SentenceTransformer('all-MiniLM-L6-v2')
        
        # Generate embeddings for known full forms
        self.known_full_forms = known_full_forms
        self.known_embeddings = self.model.encode(known_full_forms)
        
        # Store context examples if provided
        self.context_examples = context_examples or {}
        self.context_embeddings = {}
        
        # Set default thresholds
        self.threshold_high = 0.9
        self.threshold_low = 0.6
        
        # Generate embeddings for context examples
        for context, examples in self.context_examples.items():
            if examples:  # Check if there are examples for this context
                self.context_embeddings[context] = self.model.encode(" ".join(examples))
    
    def match(self, acronym, context=None):
        """
        Match an acronym using embedding-based semantic similarity.
        
        Args:
            acronym (str): The acronym to match.
            context (str, optional): Context to help disambiguation.
            
        Returns:
            tuple: (matched_full_form, confidence_score, confidence_level)
        """
        # Generate embedding for the acronym
        acronym_embedding = self.model.encode([acronym])[0].reshape(1, -1)
        
        # Calculate similarity with all known full forms
        similarities = cosine_similarity(acronym_embedding, self.known_embeddings)
        best_idx = np.argmax(similarities)
        best_score = similarities[0][best_idx]
        best_match = self.known_full_forms[best_idx]
        
        # If context is provided, adjust the matching based on context
        if context and context in self.context_embeddings:
            context_embedding = self.context_embeddings[context].reshape(1, -1)
            context_similarities = cosine_similarity(acronym_embedding, context_embedding)
            context_score = context_similarities[0][0]
            
            # If context is strongly relevant, adjust the matching
            if context_score > 0.7:
                # Find full forms that are contextually relevant
                context_relevant_forms = []
                for i, full_form in enumerate(self.known_full_forms):
                    full_form_embedding = self.known_embeddings[i].reshape(1, -1)
                    relevance = cosine_similarity(full_form_embedding, context_embedding)[0][0]
                    if relevance > 0.7:
                        context_relevant_forms.append((full_form, similarities[0][i], relevance))
                
                # If we have context-relevant forms, choose the best one
                if context_relevant_forms:
                    # Sort by combined score (similarity to acronym * relevance to context)
                    context_relevant_forms.sort(key=lambda x: x[1] * x[2], reverse=True)
                    best_match = context_relevant_forms[0][0]
                    best_score = context_relevant_forms[0][1]  # Using just similarity to acronym as score
        
        # Decision based on confidence thresholds
        if best_score >= self.threshold_high:
            confidence = "high"
        elif best_score >= self.threshold_low:
            confidence = "medium"
        else:
            confidence = "low"
            
        return best_match, best_score, confidence

#------------------------------------------------------------------------------
# HYBRID APPROACH
#------------------------------------------------------------------------------

class HybridAcronymMatcher:
    def __init__(self, known_full_forms, context_examples=None):
        """
        Initialize the hybrid acronym matcher that combines rule-based, fuzzy, and embedding-based matching.
        
        Args:
            known_full_forms (list): List of known full forms to match against.
            context_examples (dict, optional): Dictionary mapping contexts to lists of examples.
        """
        # Initialize the three matching components
        self.rule_based = RuleBasedMatcher()
        self.fuzzy = FuzzyMatcher(known_full_forms)
        self.embedding = EmbeddingMatcher(known_full_forms, context_examples)
        
        # Keep track of all known full forms
        self.known_full_forms = known_full_forms
    
    def match(self, acronym, context=None):
        """
        Perform hybrid matching on the given acronym.
        
        Args:
            acronym (str): The acronym to match.
            context (str, optional): Context information for disambiguation.
            
        Returns:
            tuple: (matched_full_form, confidence_score, method_used, decision)
        """
        # Step 1: Try rule-based matching first (fastest)
        rule_match, rule_score = self.rule_based.match(acronym)
        
        if rule_match:
            return rule_match, rule_score, "rule_based", "accept"
        
        # Step 2: If rule-based fails, try fuzzy matching
        fuzzy_match, fuzzy_score, fuzzy_confidence = self.fuzzy.match(acronym)
        
        # Step 3: If fuzzy matching gives high confidence, use it
        if fuzzy_confidence == "high":
            return fuzzy_match, fuzzy_score, "fuzzy", "accept"
        
        # Step 4: If fuzzy matching is uncertain, try embedding-based matching
        embedding_match, embedding_score, embedding_confidence = self.embedding.match(acronym, context)
        
        # Step 5: Make a decision based on confidence scores
        if embedding_confidence == "high":
            return embedding_match, embedding_score, "embedding", "accept"
        
        # Step 6: If both fuzzy and embedding have medium confidence, choose the one with higher score
        if fuzzy_confidence == "medium" and embedding_confidence == "medium":
            if fuzzy_score >= embedding_score:
                return fuzzy_match, fuzzy_score, "fuzzy", "review"
            else:
                return embedding_match, embedding_score, "embedding", "review"
        
        # Step 7: If one has medium confidence and the other has low, choose the medium one
        if fuzzy_confidence == "medium":
            return fuzzy_match, fuzzy_score, "fuzzy", "review"
        
        if embedding_confidence == "medium":
            return embedding_match, embedding_score, "embedding", "review"
        
        # Step 8: If both have low confidence, reject
        return "No match", max(fuzzy_score, embedding_score), "hybrid", "reject"
    
    def add_known_full_form(self, full_form):
        """Add a new full form to the known full forms list."""
        if full_form not in self.known_full_forms:
            self.known_full_forms.append(full_form)
            # Update the fuzzy and embedding matchers
            self.fuzzy = FuzzyMatcher(self.known_full_forms)
            self.embedding = EmbeddingMatcher(self.known_full_forms, self.embedding.context_examples)

#------------------------------------------------------------------------------
# ALTERNATIVE IMPLEMENTATIONS OF COMPARISON ALGORITHMS
#------------------------------------------------------------------------------

def jaro_winkler_similarity_match(acronym, full_forms):
    """
    Match an acronym using Jaro-Winkler similarity.
    
    Args:
        acronym (str): The acronym to match.
        full_forms (list): List of full forms to match against.
        
    Returns:
        tuple: (best_match, best_score)
    """
    best_match = None
    best_score = 0
    
    for full_form in full_forms:
        score = jaro_winkler_similarity(acronym.lower(), full_form.lower())
        if score > best_score:
            best_match = full_form
            best_score = score
    
    return best_match, best_score

def damerau_levenshtein_similarity_match(acronym, full_forms):
    """
    Match an acronym using Damerau-Levenshtein similarity (handles transpositions).
    
    Args:
        acronym (str): The acronym to match.
        full_forms (list): List of full forms to match against.
        
    Returns:
        tuple: (best_match, best_score)
    """
    best_match = None
    best_score = 0
    
    for full_form in full_forms:
        # Damerau-Levenshtein distance considers transpositions
        distance = Levenshtein.distance(acronym.lower(), full_form.lower())
        max_len = max(len(acronym), len(full_form))
        score = 1 - (distance / max_len)  # Normalize to [0, 1]
        
        if score > best_score:
            best_match = full_form
            best_score = score
    
    return best_match, best_score

def tfidf_cosine_similarity_match(acronym, full_forms):
    """
    Match an acronym using TF-IDF cosine similarity.
    
    Args:
        acronym (str): The acronym to match.
        full_forms (list): List of full forms to match against.
        
    Returns:
        tuple: (best_match, best_score)
    """
    from sklearn.feature_extraction.text import TfidfVectorizer
    
    # Create a corpus with the acronym and all full forms
    corpus = [acronym] + full_forms
    
    # Create TF-IDF vectorizer
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(corpus)
    
    # Compute cosine similarity between acronym and full forms
    cosine_similarities = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:]).flatten()
    
    # Find the best match
    best_idx = np.argmax(cosine_similarities)
    best_score = cosine_similarities[best_idx]
    best_match = full_forms[best_idx]
    
    return best_match, best_score

def jaccard_bigram_similarity_match(acronym, full_forms):
    """
    Match an acronym using Jaccard similarity on character bigrams.
    
    Args:
        acronym (str): The acronym to match.
        full_forms (list): List of full forms to match against.
        
    Returns:
        tuple: (best_match, best_score)
    """
    def get_bigrams(text):
        return [text[i:i+2] for i in range(len(text)-1)]
    
    def jaccard_similarity(set1, set2):
        intersection = len(set1.intersection(set2))
        union = len(set1) + len(set2) - intersection
        return intersection / union if union > 0 else 0
    
    best_match = None
    best_score = 0
    
    # Get bigrams for the acronym
    acronym_bigrams = set(get_bigrams(acronym.lower()))
    
    for full_form in full_forms:
        # Get bigrams for the full form
        full_form_bigrams = set(get_bigrams(full_form.lower()))
        
        # Calculate Jaccard similarity
        score = jaccard_similarity(acronym_bigrams, full_form_bigrams)
        
        if score > best_score:
            best_match = full_form
            best_score = score
    
    return best_match, best_score

def soundex_match(acronym, full_forms):
    """
    Match an acronym using Soundex phonetic matching.
    
    Args:
        acronym (str): The acronym to match.
        full_forms (list): List of full forms to match against.
        
    Returns:
        tuple: (best_match, best_score)
    """
    best_match = None
    best_score = 0
    
    acronym_soundex = soundex(acronym)
    
    for full_form in full_forms:
        # For multi-word full forms, check each word
        words = full_form.split()
        
        # Check if any word's soundex matches the acronym's soundex
        soundex_matches = [1 if soundex(word) == acronym_soundex else 0 for word in words]
        
        # If any word matches, score is 1, otherwise 0
        score = 1 if any(soundex_matches) else 0
        
        if score > best_score:
            best_match = full_form
            best_score = score
    
    return best_match, best_score

def token_sort_ratio_match(acronym, full_forms):
    """
    Match an acronym using token sort ratio (handles word order differences).
    
    Args:
        acronym (str): The acronym to match.
        full_forms (list): List of full forms to match against.
        
    Returns:
        tuple: (best_match, best_score)
    """
    best_match = None
    best_score = 0
    
    for full_form in full_forms:
        # Calculate token sort ratio
        score = fuzz.token_sort_ratio(acronym, full_form) / 100.0
        
        if score > best_score:
            best_match = full_form
            best_score = score
    
    return best_match, best_score

def contains_ratio_match(acronym, full_forms):
    """
    Match an acronym based on whether it's contained in a full form.
    
    Args:
        acronym (str): The acronym to match.
        full_forms (list): List of full forms to match against.
        
    Returns:
        tuple: (best_match, best_score)
    """
    best_match = None
    best_score = 0
    
    for full_form in full_forms:
        # Check if acronym is contained in full form
        if acronym.lower() in full_form.lower():
            # Calculate ratio of acronym length to full form length
            score = len(acronym) / len(full_form)
        else:
            score = 0
        
        if score > best_score:
            best_match = full_form
            best_score = score
    
    return best_match, best_score

def fuzzy_levenshtein_match(acronym, full_forms):
    """
    Match an acronym using difflib's SequenceMatcher (similar to Levenshtein).
    
    Args:
        acronym (str): The acronym to match.
        full_forms (list): List of full forms to match against.
        
    Returns:
        tuple: (best_match, best_score)
    """
    best_match = None
    best_score = 0
    
    for full_form in full_forms:
        # Calculate similarity using difflib's SequenceMatcher
        score = difflib.SequenceMatcher(None, acronym.lower(), full_form.lower()).ratio()
        
        if score > best_score:
            best_match = full_form
            best_score = score
    
    return best_match, best_score

def trie_approximate_match(acronym, full_forms):
    """
    A simplified version of trie approximate matching that uses prefix matching.
    
    Args:
        acronym (str): The acronym to match.
        full_forms (list): List of full forms to match against.
        
    Returns:
        tuple: (best_match, best_score)
    """
    best_match = None
    best_score = 0
    
    # Lowercase the acronym for comparison
    acronym_lower = acronym.lower()
    
    # Try exact match first
    for full_form in full_forms:
        if acronym_lower == full_form.lower():
            return full_form, 1.0
    
    # Try prefix matches
    for i in range(1, len(acronym) + 1):
        prefix = acronym[:i].lower()
        
        # Find all full forms that start with this prefix
        matches = []
        for full_form in full_forms:
            if full_form.lower().startswith(prefix):
                matches.append((full_form, i / len(acronym)))
        
        if matches:
            # Find the best match by length of matching prefix
            best_prefix_match = max(matches, key=lambda x: x[1])
            
            if best_prefix_match[1] > best_score:
                best_match = best_prefix_match[0]
                best_score = best_prefix_match[1]
    
    return best_match, best_score

def embedding_similarity_match(acronym, full_forms):
    """
    Match an acronym using embedding-based semantic similarity.
    
    Args:
        acronym (str): The acronym to match.
        full_forms (list): List of full forms to match against.
        
    Returns:
        tuple: (best_match, best_score)
    """
    # Use sentence transformers for embeddings
    model = SentenceTransformer('all-MiniLM-L6-v2')
    
    # Generate embeddings
    acronym_embedding = model.encode([acronym])[0].reshape(1, -1)
    full_form_embeddings = model.encode(full_forms)
    
    # Calculate cosine similarity
    similarities = cosine_similarity(acronym_embedding, full_form_embeddings).flatten()
    
    # Find the best match
    best_idx = np.argmax(similarities)
    best_score = similarities[best_idx]
    best_match = full_forms[best_idx]
    
    return best_match, best_score

def aho_corasick_match(acronym, full_forms):
    """
    A simplified version of Aho-Corasick matching that uses word-level matching.
    
    Args:
        acronym (str): The acronym to match.
        full_forms (list): List of full forms to match against.
        
    Returns:
        tuple: (best_match, best_score)
    """
    best_match = None
    best_score = 0
    
    # Split acronym into words and lowercase
    acronym_words = set(acronym.lower().split())
    
    # Count word matches for each full form
    match_counts = []
    
    for full_form in full_forms:
        # Split full form into words and lowercase
        full_form_words = set(full_form.lower().split())
        
        # Count matching words
        matches = len(acronym_words.intersection(full_form_words))
        total_words = len(acronym_words)
        
        if total_words > 0:
            score = matches / total_words
            match_counts.append((full_form, score))
    
    # Find the full form with the highest score
    if match_counts:
        match_counts.sort(key=lambda x: x[1], reverse=True)
        best_match, best_score = match_counts[0]
    
    return best_match, best_score

#------------------------------------------------------------------------------
# EVALUATION FUNCTIONS
#------------------------------------------------------------------------------

def evaluate_algorithms(test_data, full_forms):
    """
    Evaluate the performance of all algorithms on the test data.
    
    Args:
        test_data (list): List of tuples (acronym, correct_full_form, context).
        full_forms (list): List of all possible full forms.
        
    Returns:
        dict: Dictionary of algorithm names and their performance metrics.
    """
    algorithms = {
        "Hybrid": None,  # We'll initialize the hybrid matcher separately
        "Jaro-Winkler": jaro_winkler_similarity_match,
        "Damerau-Levenshtein": damerau_levenshtein_similarity_match,
        "TF-IDF Cosine": tfidf_cosine_similarity_match,
        "Jaccard Bigram": jaccard_bigram_similarity_match,
        "Soundex": soundex_match,
        "Token Sort Ratio": token_sort_ratio_match,
        "Contains Ratio": contains_ratio_match,
        "Fuzzy Levenshtein": fuzzy_levenshtein_match,
        "Trie Approximate": trie_approximate_match,
        "Embedding Similarity": embedding_similarity_match,
        "Aho-Corasick": aho_corasick_match
    }
    
    # Context examples for the embedding matcher
    context_examples = defaultdict(list)
    for acronym, full_form, context in test_data:
        if context:
            context_examples[context].append(full_form)
    
    # Initialize the hybrid matcher
    hybrid_matcher = HybridAcronymMatcher(full_forms, context_examples)
    
    # Prepare results dictionary
    results = {name: {"correct": 0, "total": 0, "predictions": []} for name in algorithms}
    
    # Evaluate each algorithm
    print(f"Evaluating {len(algorithms)} algorithms on {len(test_data)} test cases...")
    
    for i, (acronym, correct_full_form, context) in enumerate(test_data):
        if i % 20 == 0 and i > 0:
            print(f"Processed {i}/{len(test_data)} test cases...")
            
        # Evaluate hybrid matcher
        hybrid_match, hybrid_score, method, decision = hybrid_matcher.match(acronym, context)
        results["Hybrid"]["predictions"].append((acronym, correct_full_form, hybrid_match, hybrid_score))
        if hybrid_match == correct_full_form:
            results["Hybrid"]["correct"] += 1
        results["Hybrid"]["total"] += 1
        
        # Evaluate other algorithms
        for name, algorithm in algorithms.items():
            if name == "Hybrid":
                continue
                
            match, score = algorithm(acronym, full_forms)
            results[name]["predictions"].append((acronym, correct_full_form, match, score))
            if match == correct_full_form:
                results[name]["correct"] += 1
            results[name]["total"] += 1
    
    # Calculate accuracy, precision, recall, and F1 scores
    for name in algorithms:
        correct = results[name]["correct"]
        total = results[name]["total"]
        accuracy = correct / total if total > 0 else 0
        results[name]["accuracy"] = accuracy
        
        # Calculate additional metrics if needed
        predictions = [(p[2] == p[1]) for p in results[name]["predictions"]]
        true_labels = [True] * len(predictions)  # All should be True for perfect matching
        
        # Calculate precision, recall, and F1 score
        if sum(predictions) > 0:  # Avoid division by zero
            precision = precision_score(true_labels, predictions, zero_division=0)
            recall = recall_score(true_labels, predictions, zero_division=0)
            f1 = f1_score(true_labels, predictions, zero_division=0)
        else:
            precision = recall = f1 = 0
            
        results[name]["precision"] = precision
        results[name]["recall"] = recall
        results[name]["f1"] = f1
    
    return results

def write_results_to_csv(results, output_file):
    """
    Write the evaluation results to a CSV file.
    
    Args:
        results (dict): Dictionary of algorithm names and their performance metrics.
        output_file (str): Path to the output CSV file.
    """
    # Prepare CSV header
    header = ["Algorithm", "Accuracy", "Precision", "Recall", "F1-Score", "Correct", "Total"]
    
    # Write results to CSV
    with open(output_file, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(header)
        
        # Sort algorithms by accuracy (descending)
        sorted_algorithms = sorted(results.keys(), key=lambda x: results[x]["accuracy"], reverse=True)
        
        # Write results for each algorithm
        for algorithm in sorted_algorithms:
            writer.writerow([
                algorithm,
                f"{results[algorithm]['accuracy']:.4f}",
                f"{results[algorithm]['precision']:.4f}",
                f"{results[algorithm]['recall']:.4f}",
                f"{results[algorithm]['f1']:.4f}",
                results[algorithm]["correct"],
                results[algorithm]["total"]
            ])
        
    print(f"Results written to {output_file}")

def write_predictions_to_csv(results, output_file):
    """
    Write the algorithm predictions to a CSV file.
    
    Args:
        results (dict): Dictionary of algorithm names and their performance metrics.
        output_file (str): Path to the output CSV file.
    """
    # Get all unique acronyms from the first algorithm's predictions
    algorithm = list(results.keys())[0]
    acronyms = [p[0] for p in results[algorithm]["predictions"]]
    correct_forms = [p[1] for p in results[algorithm]["predictions"]]
    
    # Prepare CSV header
    header = ["Acronym", "Correct Full Form"] + [f"{algo}_Match" for algo in results] + [f"{algo}_Score" for algo in results]
    
    # Write predictions to CSV
    with open(output_file, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(header)
        
        # Write predictions for each acronym
        for i, acronym in enumerate(acronyms):
            row = [acronym, correct_forms[i]]
            
            # Add predictions from each algorithm
            for algorithm in results:
                prediction = results[algorithm]["predictions"][i]
                row.append(prediction[2])  # Match
            
            # Add scores from each algorithm
            for algorithm in results:
                prediction = results[algorithm]["predictions"][i]
                row.append(f"{prediction[3]:.4f}")  # Score
            
            writer.writerow(row)
        
    print(f"Predictions written to {output_file}")

#------------------------------------------------------------------------------
# THRESHOLD OPTIMIZATION
#------------------------------------------------------------------------------

def optimize_thresholds(test_data, full_forms):
    """
    Optimize the thresholds for the hybrid approach to maximize accuracy.
    
    Args:
        test_data (list): List of tuples (acronym, correct_full_form, context).
        full_forms (list): List of all possible full forms.
        
    Returns:
        tuple: Optimized thresholds (fuzzy_high, fuzzy_low, embedding_high, embedding_low).
    """
    print("Starting threshold optimization...")
    
    # Context examples for the embedding matcher
    context_examples = defaultdict(list)
    for acronym, full_form, context in test_data:
        if context:
            context_examples[context].append(full_form)
    
    # Define threshold ranges to search (using fewer values for faster optimization)
    fuzzy_high_range = [0.75, 0.8, 0.85, 0.9]
    fuzzy_low_range = [0.5, 0.55, 0.6, 0.65, 0.7]
    embedding_high_range = [0.75, 0.8, 0.85, 0.9]
    embedding_low_range = [0.5, 0.55, 0.6, 0.65, 0.7]
    
    best_accuracy = 0
    best_thresholds = (0.85, 0.6, 0.9, 0.6)  # Default values in case optimization fails
    
    # For faster optimization, limit the number of combinations tested
    max_combinations = 25
    tested = 0
    
    print(f"Testing up to {max_combinations} threshold combinations...")
    
    try:
        for fuzzy_high in fuzzy_high_range:
            if tested >= max_combinations:
                break
                
            for fuzzy_low in fuzzy_low_range:
                if fuzzy_low >= fuzzy_high or tested >= max_combinations:
                    continue
                    
                for embedding_high in embedding_high_range:
                    if tested >= max_combinations:
                        break
                        
                    for embedding_low in embedding_low_range:
                        if embedding_low >= embedding_high:
                            continue
                            
                        tested += 1
                        if tested > max_combinations:
                            break
                        
                        print(f"Testing combination {tested}: [{fuzzy_high}, {fuzzy_low}, {embedding_high}, {embedding_low}]", end="\r")
                        
                        # Initialize the hybrid matcher with the current thresholds
                        hybrid_matcher = HybridAcronymMatcher(full_forms, context_examples)
                        
                        # Override the default thresholds for testing
                        hybrid_matcher.fuzzy.threshold_high = fuzzy_high
                        hybrid_matcher.fuzzy.threshold_low = fuzzy_low
                        hybrid_matcher.embedding.threshold_high = embedding_high
                        hybrid_matcher.embedding.threshold_low = embedding_low
                        
                        # Evaluate the matcher on the test data
                        correct = 0
                        total = len(test_data)
                        
                        for acronym, correct_full_form, context in test_data:
                            match, _, _, _ = hybrid_matcher.match(acronym, context)
                            if match == correct_full_form:
                                correct += 1
                        
                        # Calculate accuracy
                        accuracy = correct / total
                        
                        # Update best thresholds if accuracy improves
                        if accuracy > best_accuracy:
                            best_accuracy = accuracy
                            best_thresholds = (fuzzy_high, fuzzy_low, embedding_high, embedding_low)
                            
                            print(f"\nFound better thresholds: {best_thresholds} with accuracy: {best_accuracy:.4f}")
    
    except Exception as e:
        print(f"\nError during threshold optimization: {str(e)}")
        print("Using default thresholds instead.")
    
    print(f"\nFinal best thresholds: {best_thresholds} with accuracy: {best_accuracy:.4f}")
    return best_thresholds

#------------------------------------------------------------------------------
# MAIN EXECUTION
#------------------------------------------------------------------------------

def run_prototype(excel_file_path="Acronym.xlsx"):
    """
    Run the complete prototype implementation using data from an Excel file.
    
    Args:
        excel_file_path (str): Path to the Excel file containing acronym data.
        
    Returns:
        HybridAcronymMatcher: The optimized hybrid matcher.
    """
    # Load data from Excel file
    full_forms, test_data = load_data_from_excel(excel_file_path)
    
    # Check if data was loaded successfully
    if not full_forms or not test_data:
        print("No data loaded from Excel. Using example data instead.")
        full_forms, test_data = create_example_data()
    
    # Optimize thresholds for the hybrid approach
    print("Optimizing thresholds for the hybrid approach...")
    best_thresholds = optimize_thresholds(test_data, full_forms)
    
    # Context examples for the embedding matcher
    context_examples = defaultdict(list)
    for acronym, full_form, context in test_data:
        if context:
            context_examples[context].append(full_form)
    
    # Initialize the hybrid matcher with optimized thresholds
    print("Initializing hybrid matcher with optimized thresholds...")
    hybrid_matcher = HybridAcronymMatcher(full_forms, context_examples)
    
    # Override the default thresholds with the optimized ones
    fuzzy_high, fuzzy_low, embedding_high, embedding_low = best_thresholds
    hybrid_matcher.fuzzy.threshold_high = fuzzy_high
    hybrid_matcher.fuzzy.threshold_low = fuzzy_low
    hybrid_matcher.embedding.threshold_high = embedding_high
    hybrid_matcher.embedding.threshold_low = embedding_low
    
    # Evaluate all algorithms including the optimized hybrid approach
    print("\nEvaluating all algorithms...")
    results = evaluate_algorithms(test_data, full_forms)
    
    # Print accuracy scores
    print("\nAccuracy Scores:")
    for algorithm, metrics in sorted(results.items(), key=lambda x: x[1]["accuracy"], reverse=True):
        print(f"{algorithm}: {metrics['accuracy']:.4f}, F1: {metrics['f1']:.4f}")
    
    # Write results to CSV
    write_results_to_csv(results, "acronym_matching_results.csv")
    write_predictions_to_csv(results, "acronym_matching_predictions.csv")
    
    print("\nResults have been written to CSV files.")
    
    # Return the hybrid matcher for potential use
    return hybrid_matcher

if __name__ == "__main__":
    print("Starting Hybrid Acronym Matcher prototype with data from Acronym.xlsx...")
    hybrid_matcher = run_prototype("Acronym.xlsx")
    
    print("\nExample matches using the optimized hybrid matcher:")
    test_acronyms = ["BofA", "7-11", "StarBucks", "MCD", "WLMRT", "CBA", "IBM", "ACCC"]
    
    for acronym in test_acronyms:
        match, score, method, decision = hybrid_matcher.match(acronym)
        print(f"Acronym: {acronym}")
        print(f"Matched to: {match}")
        print(f"Confidence: {score:.4f}")
        print(f"Method used: {method}")
        print(f"Decision: {decision}")
        print("-" * 40)
    
    print("Prototype execution completed.")

Starting Hybrid Acronym Matcher prototype with data from Acronym.xlsx...
Loading data from Acronym.xlsx...
Loaded 85 unique full forms and 98 test cases.
Optimizing thresholds for the hybrid approach...
Starting threshold optimization...
Testing up to 25 threshold combinations...
Testing combination 1: [0.75, 0.5, 0.75, 0.5]
Found better thresholds: (0.75, 0.5, 0.75, 0.5) with accuracy: 0.5510
Testing combination 25: [0.75, 0.55, 0.75, 0.7]]
Final best thresholds: (0.75, 0.5, 0.75, 0.5) with accuracy: 0.5510
Initializing hybrid matcher with optimized thresholds...

Evaluating all algorithms...
Evaluating 12 algorithms on 98 test cases...
Processed 20/98 test cases...
Processed 40/98 test cases...
Processed 60/98 test cases...
Processed 80/98 test cases...

Accuracy Scores:
Hybrid: 0.5510, F1: 0.7105
Jaro-Winkler: 0.3673, F1: 0.5373
Embedding Similarity: 0.3673, F1: 0.5373
Trie Approximate: 0.3061, F1: 0.4687
Damerau-Levenshtein: 0.2857, F1: 0.4444
Fuzzy Levenshtein: 0.2857, F1: 0.4444
